In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ECMA 31330 Final Project'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
sys.path.append(GOOGLE_DRIVE_PATH)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['Articles', 'SentimentScores', 'Aakash Gupta Final Project Presentation.gslides', 'Sp500Stocks.csv', 'SPY2015.csv', 'SPY2020.csv', 'SPY-2015_2020.csv', 'DataCleaningPipeline.ipynb', 'SentimentAnalysisPipeline.ipynb', 'SimpleGARCHModel.ipynb', 'GARCHFittingPipeline.ipynb', 'LassoModel.ipynb', 'NeuralNet.ipynb']


In [ ]:
targeted_path = GOOGLE_DRIVE_PATH + "/Articles"
csv_files = [file for file in os.listdir(targeted_path) if file.startswith('filtered')]
dataframes = []
for file in csv_files:
    file_path = os.path.join(targeted_path, file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)
combined_df['date'] = combined_df['date'].astype('datetime64[ns]')
combined_df = combined_df.groupby('date')['title'].agg(" ".join).reset_index()


In [ ]:
listofwords = []
listoftitles = list(combined_df['title'])
for title in listoftitles:
  listofwords.append(title.split())


flatten_listofwords = []
for xs in listofwords:
    for x in xs:
        flatten_listofwords.append(x)

flatten_listofwords = np.array(flatten_listofwords)
unique_words = np.unique(flatten_listofwords)
unique_words = [word.lower() for word in unique_words]

In [ ]:
def intialize_dictionary(unique_words):
  word_dictionary = {}
  for word in unique_words:
    word_dictionary[word] = []
  return word_dictionary

def word_populator(index, row, word_dictionary):
  title = list(row['title'])
  title = title[0].lower()
  words = title.split()
  for word in words:
    word_dictionary[word].append(index)

In [ ]:
word_dictionary = intialize_dictionary(unique_words)
for i in range(len(combined_df)):
  word_populator(i, combined_df[i:i+1], word_dictionary)

In [ ]:
word_df = np.zeros([len(combined_df), len(unique_words)])
for key in word_dictionary.keys():
  idxs = word_dictionary[key]
  for idx in idxs:
    word_df[idx][unique_words.index(key)] = 1

In [ ]:
word_df = pd.DataFrame(word_df, columns=unique_words)
final_df = pd.concat([combined_df, word_df], axis=1)

In [ ]:
stocks = pd.read_csv(GOOGLE_DRIVE_PATH + "/SPY-2015_2020.csv")
stocks = stocks[2:]
stocks["Return"] = 100 * pd.to_numeric(stocks.Close).pct_change()
stocks['date'] = stocks['Price'].astype('datetime64[ns]')
stocks["Volatility"] = abs(pd.to_numeric(stocks.Close) - pd.to_numeric(stocks.Open))
lasso_data = pd.merge(stocks, final_df, on = 'date', how = 'inner')
lasso_data.head()

,Price,Close,High,Low,Open,Volume,Return,date,Volatility,title,...,zoomlion,zooms,zouping,zte,zuckerberg,zuckerbergs,zuluan,zuying,zwarte,zwirner
0,2015-01-02,172.59291076660156,173.8111448920031,171.54271777555792,173.39106769558566,121465900,-0.053521,2015-01-02,0.798157,stig bergling a cold war spy known for his esc...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-01-05,169.4759063720703,171.70230987578518,169.16505342016106,171.5342815996109,169632600,-1.805986,2015-01-05,2.058375,in iran fatal porsche crash unleashes middle c...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-01-06,167.87962341308594,170.3160655379834,167.0730697163054,169.78676427385443,209151400,-0.941894,2015-01-06,1.907141,ex fifa official jack warner cites onion artic...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015-01-07,169.97158813476562,170.31605442130996,168.77017382476237,169.2238514360209,125346700,1.246110,2015-01-07,0.747737,pressed by debt crisis doctors leave greece in...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-01-08,172.98773193359375,173.2061802141637,171.38304727257352,171.3998411251291,147217800,1.774499,2015-01-08,1.587891,greek debt crisis adds to a spike in burglarie...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
lasso_data['Volatility_1'] = lasso_data["Volatility"].shift(1)
lasso_data['Volatility_2'] = lasso_data["Volatility"].shift(2)
lasso_data['Volatility_1'] = scaler.fit_transform(lasso_data[['Volatility_1']])
lasso_data['Volatility_2'] = scaler.fit_transform(lasso_data[['Volatility_2']])
lasso_data = lasso_data.dropna()
lasso_data.head()

,Price,Close,High,Low,Open,Volume,Return,date,Volatility,title,...,zouping,zte,zuckerberg,zuckerbergs,zuluan,zuying,zwarte,zwirner,Volatility_1,Volatility_2
2,2015-01-06,167.87962341308594,170.3160655379834,167.0730697163054,169.78676427385443,209151400,-0.941894,2015-01-06,1.907141,ex fifa official jack warner cites onion artic...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.609036,-0.254018
3,2015-01-07,169.97158813476562,170.31605442130996,168.77017382476237,169.2238514360209,125346700,1.246110,2015-01-07,0.747737,pressed by debt crisis doctors leave greece in...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.505398,0.609439
4,2015-01-08,172.98773193359375,173.2061802141637,171.38304727257352,171.3998411251291,147217800,1.774499,2015-01-08,1.587891,greek debt crisis adds to a spike in burglarie...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.289119,0.505818
5,2015-01-09,171.60150146484375,173.42463460544093,170.97978271958456,173.40782793131604,158567300,-0.801346,2015-01-09,1.806326,austria migrant crisis truck most nations miss...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.286622,-0.288564
6,2015-01-12,170.2572479248047,171.89555891315098,169.64393893239904,171.73592755984657,144396100,-0.783358,2015-01-12,1.478680,china aids sexual transmission russia moscow d...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.436312,0.287079


In [ ]:
y = np.array(lasso_data['Volatility'])
x = np.array(lasso_data.drop(["Price", "Close", "High", "Low", "Open", "Volume", "title", "date", "Return", "Volatility"], axis = 1))

In [ ]:
lasso_data.head()

,Price,Close,High,Low,Open,Volume,Return,date,Volatility,title,...,zouping,zte,zuckerberg,zuckerbergs,zuluan,zuying,zwarte,zwirner,Volatility_1,Volatility_2
2,2015-01-06,167.87962341308594,170.3160655379834,167.0730697163054,169.78676427385443,209151400,-0.941894,2015-01-06,1.907141,ex fifa official jack warner cites onion artic...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.609036,-0.254018
3,2015-01-07,169.97158813476562,170.31605442130996,168.77017382476237,169.2238514360209,125346700,1.246110,2015-01-07,0.747737,pressed by debt crisis doctors leave greece in...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.505398,0.609439
4,2015-01-08,172.98773193359375,173.2061802141637,171.38304727257352,171.3998411251291,147217800,1.774499,2015-01-08,1.587891,greek debt crisis adds to a spike in burglarie...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.289119,0.505818
5,2015-01-09,171.60150146484375,173.42463460544093,170.97978271958456,173.40782793131604,158567300,-0.801346,2015-01-09,1.806326,austria migrant crisis truck most nations miss...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.286622,-0.288564
6,2015-01-12,170.2572479248047,171.89555891315098,169.64393893239904,171.73592755984657,144396100,-0.783358,2015-01-12,1.478680,china aids sexual transmission russia moscow d...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.436312,0.287079


In [ ]:
X_train, y_train = torch.tensor(x[:-150], dtype=torch.float32), torch.tensor(y[:-150], dtype=torch.float32)
X_test, y_test = torch.tensor(x[-150:], dtype=torch.float32), torch.tensor(y[-150:], dtype=torch.float32)
batch_size = 32
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
X_train = np.array(lasso_data[:-300].drop(["Price", "Close", "High", "Low", "Open", "Volume", "title", "date", "Return", "Volatility"], axis = 1))  # Covariates
y_train = np.array(lasso_data[:-300]['Volatility'])  # Volatility target
X_test = np.array(lasso_data[-300:].drop(["Price", "Close", "High", "Low", "Open", "Volume", "title", "date", "Return", "Volatility"], axis = 1))
y_test = np.array(lasso_data[-300:]['Volatility'])
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
y_train = np.reshape(y_train, (y_train.shape[0], 1))
y_test = np.reshape(y_test, (y_test.shape[0], 1))

In [ ]:
num_samples = 1422 - 300
feature_dim = 10764

X = torch.tensor(X_train, dtype=torch.float32)
y = torch.tensor(y_train, dtype=torch.float32)

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size=192):
        super(LSTM, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        return self.fc2(x)

model = LSTM(input_size=feature_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50
batch_size = 32
dataset = torch.utils.data.TensorDataset(X, y)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()
        y_pred = model(batch_X)
        batch_y = batch_y.reshape(-1,1)

        loss = criterion(y_pred, batch_y)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

# Predictions
with torch.no_grad():
    predictions = model(torch.tensor(X_test,dtype = torch.float32)).numpy()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([2, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 10, Loss: 4.1220
Epoch 20, Loss: 0.7116
Epoch 30, Loss: 0.4493
Epoch 40, Loss: 0.0055
Epoch 50, Loss: 0.4729


In [ ]:
model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient computation
    y_train_pred = model(X)  # Predictions on training data
    y_test_pred = model(torch.tensor(X_test, dtype=torch.float32))  # Predictions on test data

    # Ensure shape consistency
    y_train = y.view(-1, 1)  # Make sure it's (num_samples, 1)
    y_test = torch.tensor(X_test, dtype=torch.float32)
    #y_test = y_test.view(-1, 1)

    # Compute MSE
    train_mse = torch.mean((y_train_pred - y_train) ** 2).item()
    test_mse = torch.mean((y_test_pred - y_test) ** 2).item()

print(f"Training MSE: {train_mse:.4f}")
print(f"Test MSE: {test_mse:.4f}")

Training MSE: 1.2472
Test MSE: 0.9554
